# Sudoku Solver - A Systematic Approach
Author : Naveen Chakravarthy Balasubramanian

### **Imports and Data**
Import necessary libraries and load the data

In [ ]:
import pandas as pd
from collections import Counter
sudokus = pd.read_csv('../input/sudoku/sudoku.csv')

### **Sudoku Solver Class**
A systematic and simple approach to solve sudoku

In [ ]:
class Sudoku():
    """Attempts to solve the sudoku in a systematic approach."""

    def __init__(self):
        pass

    def init_board(self):
        """Map the values to board positions. If value is 0, map all possible digits to that position. 
        These are to be later removed iteratively according to constraints."""
        i = 0
        for row in C_Rows:
            for col in C_Cols:
                if C_Unsolved[i] == '0':
                    C_Boxes[row + col] = '123456789'
                else:
                    C_Boxes[row + col] = C_Unsolved[i]
                i += 1
        del C_Boxes['X']

    def board_match(self, X, Y):
        """Match board rows and columns to return the cell."""
        return [r + c for r in X for c in Y]

    def popdigit(self, links, k, p):
        """Remove a digit from the list of possible values for that cell."""
        for ele in links[k]:
            C_Boxes[ele] = C_Boxes[ele].replace(p, '')

    def reduce(self, board_lnk, board_box):
        """If there is only one digit in a cell, remove that digit from all dependent cells - 
        be it in same row, same column or same box."""
        for cell in board_box:
            if len(C_Boxes[cell]) == 1:
                self.popdigit(board_lnk, cell, C_Boxes[cell])

    def unique(self, board_seg):
        """Ignoring cells that have already been confidently filled in a row / column / box, 
        if the other unpredicted cells have a loose digit - make the loose digit the actual prediction.
        Example: Considering a 3x3 square in the puzzle:
                                 1   2   3
                              --------------   
                           A |   9   8   7
                           B |   6   23  4
                           C |   23  5   123
        
        Here A1, A2, A3, B1, B3, C2 have been confidently predicted. Of the cells B2 and C1, 2 and 3 can occur only in those two.
        Hence it is safe to remove 2 and 3 from the list of possible numbers from C3. 
        
        """
        for seg in board_seg:
            fixnum = ''
            unqcnt = 0
            string = ''.join([C_Boxes[cell] for cell in seg if len(C_Boxes[cell]) != 1])
            stringcnts = Counter(string)
            if len(stringcnts) > 2:
                for k1, v1 in stringcnts.items():
                    if v1 == 1:
                        unqcnt += 1
                        fixnum += k1
            if unqcnt == 1:
                for cell in seg:
                    if fixnum in C_Boxes[cell]:
                        C_Boxes[cell] = fixnum

    def evaluate(self, board_seg):
        """Check the sum on each row, column and square to validate the correctness of the solution."""
        sums = 0
        for seg in board_seg:
            for cell in seg:
                sums += int(C_Boxes[cell])
        if sums == 405:
            return True
        else:
            return False

    def join_board(self):
        """Join the board as a string to compare against the solutions in the dataset."""
        solved_board = ''
        for row in C_Rows:
            for col in C_Cols:
                solved_board += C_Boxes[row + col]
        return solved_board

    def solve(self):
        """Begin solving the puzzle after defining the board and establishing dependency cells for each cell."""
        board_box = self.board_match(C_Rows, C_Cols)
        board_row = [self.board_match(R, C_Cols) for R in C_Rows]
        board_col = [self.board_match(C_Rows, C) for C in C_Cols]
        board_sqr = [self.board_match(Rs, Cs) for Rs in ('ABC', 'DEF', 'GHI') for Cs in ('123', '456', '789')]
        board_lst = board_row + board_col + board_sqr
        board_all = dict((cell, [lst for lst in board_lst if cell in lst]) for cell in board_box)
        board_lnk = dict((cell, sorted(set(sum(board_all[cell], [])) - set([cell]))) for cell in board_box)
        for i in range(5):
            for j in range(3):
                self.reduce(board_lnk, board_box)
            for board_seg in [board_sqr, board_col, board_row]:
                self.unique(board_seg)
        a = self.join_board()
        return a

    def display_board(self):
        """Display the solved board"""
        print('-' * 37)
        for row in C_Rows:
            board = '|'
            for col in C_Cols:
                board += '  ' + C_Boxes[row + col]
                if col in '369':
                    board += '  |'
            print(board)
            if row in 'CF':
                print('-' * 37)
        print('-' * 37)  
        

### **Run**

In [ ]:
C_Rows = 'ABCDEFGHI'
C_Cols = '123456789'
C_Correct = 0
game = Sudoku()
for i in range(len(sudokus)):
    C_Unsolved = sudokus.quizzes[i]
    C_Unsolved = ''.join([char for char in C_Unsolved if char != '.'])
    C_Boxes = {'X' : 0}
    solution = sudokus.solutions[i]
    game.init_board() 
    a = game.solve()
    if a == solution:
        C_Correct += 1
print(f"{C_Correct} out of {len(sudokus)} correct. Accuracy is {(C_Correct / len(sudokus)) * 100:.4f}%")